# MHC Peptide Binding Pipeline
### Supporting research: Covid19 and beyond for vaccine candidates
Cloud AI Platform Pipelines provides a way to deploy robust, repeatable machine learning pipelines along with monitoring, auditing, version tracking, and reproducibility, and delivers an enterprise-ready, easy to install, secure execution environment for your ML workflows. 
The goal is to democratize machine learning by enabling SQL practitioners to build models using their existing tools and to increase development speed by eliminating the need for data movement. In this tutorial, you use the sample Covid19 dataset for BigQuery.


## Step 1. Set up your environment.

AI Platform Pipelines will prepare a development environment to build a pipeline, and a Kubeflow Pipeline cluster to run the newly built pipeline.

**NOTE:** To select a particular TensorFlow version, or select a GPU instance, create a TensorFlow pre-installed instance in AI Platform Notebooks.

**NOTE:** There might be some errors during package installation. For example: 

>"ERROR: some-package 0.some_version.1 has requirement other-package!=2.0.,&lt;3,&gt;=1.15, but you'll have other-package 2.0.0 which is incompatible." Please ignore these errors at this moment.


Install `tfx`, `kfp`, and `skaffold`, and add installation path to the `PATH` environment variable.

In [2]:
# Install tfx and kfp Python packages.
!pip install --user --upgrade -q tfx==0.21.2
!pip install --user --upgrade -q kfp==0.2.5
# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold && mv skaffold /home/jupyter/.local/bin/

ERROR: google-cloud-storage 1.26.0 has requirement google-resumable-media<0.6dev,>=0.5.0, but you'll have google-resumable-media 0.4.1 which is incompatible.
ERROR: google-cloud-bigquery 1.17.1 has requirement google-resumable-media<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.0 which is incompatible.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 39.2M  100 39.2M    0     0  58.6M      0 --:--:-- --:--:-- --:--:-- 58.6M


In [3]:
# 1.1 Set Path

In [4]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [5]:
# 1.2 Check version

In [6]:
!python3 -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

TFX version: 0.21.2


In [7]:
# 1.3 Check GCP Project

In [8]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GCP_PROJECT_ID=shell_output[0]
print("GCP project ID:" + GCP_PROJECT_ID)

GCP project ID:covid-19-271622


We also need to access your KFP cluster. You can access it in your Google Cloud Console under "AI Platform > Pipeline" menu. The "endpoint" of the KFP cluster can be found from the URL of the Pipelines dashboard, or you can get it from the URL of the Getting Started page where you launched this notebook. Let's create an `ENDPOINT` environment variable and set it to the KFP cluster endpoint. **ENDPOINT should contain only the hostname part of the URL.** For example, if the URL of the KFP dashboard is `https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start`, ENDPOINT value becomes `1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com`.

>**NOTE: You MUST set your ENDPOINT value below.**

In [9]:
# 1.4 Set AI Pipeline Kube cluster endpoint

In [10]:
# This refers to the KFP cluster endpoint
ENDPOINT='4dfa62a617d46f32-dot-us-central2.pipelines.googleusercontent.com' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

In [11]:
# 1.5 set custom image

In [12]:
# Docker image name for the pipeline image 
CUSTOM_TFX_IMAGE='gcr.io/' + GCP_PROJECT_ID + '/tfx-pipeline'

## Step 2. Copy the predefined template to your project directory.

In this step, we will create a working pipeline project directory and files by copying additional files from a predefined template.

You may give your pipeline a **different name by changing the `PIPELINE_NAME` below.** This will also become the name of the project directory where your files will be put.

In [13]:
# 1.6 Set pipeline name

In [14]:
PIPELINE_NAME="mch_pipeline"
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"AIHub",PIPELINE_NAME)

This TFX pipeline includes the `mhc peptide` template with the TFX python package. If you are planning to solve a point-wise prediction problem, including classification and regresssion, this template could be used as a starting point.

We will be copying pre-built Peptide Prediction pipeline files from shared GCS bucket. You can also use
the `tfx template copy` CLI command copies predefined template files into your project directory.

**Note:** Create a folder called '**mhc_pipeline**' under AIHub on the left panel file browser.

In [15]:
# 1.7 run once to copy template files

In [16]:
#!tfx template copy \
#  --pipeline_name={PIPELINE_NAME} \
#  --destination_path={PROJECT_DIR} \
#  --model=taxi

In [17]:
# 2. Validate setup

In [18]:
%cd {PROJECT_DIR}

/home/jupyter/AIHub/mch_pipeline


In [19]:
# 2.1 Check files

In [20]:
!ls

beam_dag_runner.py     features.py	       model.py
build.yaml	       features_test.py        model_test.py
configs.py	       hparams.py	       old
data		       __init__.py	       pipeline.py
data_validation.ipynb  kubeflow_dag_runner.py  preprocessing.py
deployedmhc	       mch_pipeline.tar.gz     preprocessing_test.py
Dockerfile	       model_analysis.ipynb    __pycache__


In [21]:
# 2.2 Test template feature set

In [22]:
!python3 features_test.py

Running tests under Python 3.7.6: /opt/conda/bin/python3
[ RUN      ] FeaturesTest.testNumberOfBucketFeatureBucketCount
[  FAILED  ] FeaturesTest.testNumberOfBucketFeatureBucketCount
[ RUN      ] FeaturesTest.testTransformedNames
[       OK ] FeaturesTest.testTransformedNames
[ RUN      ] FeaturesTest.test_session
[  SKIPPED ] FeaturesTest.test_session
FAIL: testNumberOfBucketFeatureBucketCount (__main__.FeaturesTest)
testNumberOfBucketFeatureBucketCount (__main__.FeaturesTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "features_test.py", line 33, in testNumberOfBucketFeatureBucketCount
    len(features.CATEGORICAL_FEATURE_MAX_VALUES))
AssertionError: 1 != 0

----------------------------------------------------------------------
Ran 3 tests in 0.008s

FAILED (failures=1, skipped=1)


In [25]:
# 2.3 Review GCS bucket list

In [26]:
# You can see your buckets using `gsutil`. Following command will show bucket names without prefix and postfix.
!gsutil ls | cut -d / -f 3

artifacts.covid-19-271622.appspot.com
cancer_vaccine
dataproc-staging-us-central1-598002519658-gxbdggqa
dataproc-temp-us-central1-598002519658-hu8gmvbe
hla_peptide_dataset
hostedkfp-default-y95ed9e0de
hostedkfp-default-yfu8n9ppw9
mhc_peptide_model
mhc_pipelines
virus_mutations


In [27]:
# 3. Create pipeline based on template

In [34]:
!tfx pipeline create  \
--pipeline_path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT} \
--build_target_image={CUSTOM_TFX_IMAGE}

CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
Target image gcr.io/covid-19-271622/tfx-pipeline is not used. If the build spec is provided, update the target image in the build spec file build.yaml.
Use skaffold to build the container image.
/home/jupyter/.local/bin/skaffold
New container image is built. Target image is available in the build spec file.
/home/jupyter/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
INFO:absl:Neither eval_config nor feature_slicing_spec is passed, the model is treated as estimator.
Instructions for updating:
ModelValidator is deprecated, use Evaluator instead.
INFO:absl:Adding upstream dependencies for component BigQueryExampleGen
INFO:absl:Adding upstream dependencies for component Sta

In [29]:
# 3.1 Run an experiment on a pipeline

In [30]:
!tfx run create --pipeline_name={PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: mch_pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: mch_pipeline
+-----------------+--------------------------------------+----------+---------------------------+
| pipeline_name   | run_id                               | status   | created_at                |
+=================+======================================+==========+===========================+
| mch_pipeline    | bdb9cce8-53a8-4685-802e-efdea4f26f9d |          | 2020-04-29T18:22:06+00:00 |
+-----------------+--------------------------------------+----------+---------------------------+


In [31]:
# 4. Update pipeline and run an experiment for modifications

In [36]:
# Update the pipeline
!tfx pipeline update \
--pipeline_path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
!tfx run create --pipeline_name {PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
Use skaffold to build the container image.
/home/jupyter/.local/bin/skaffold
New container image is built. Target image is available in the build spec file.
/home/jupyter/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
INFO:absl:Neither eval_config nor feature_slicing_spec is passed, the model is treated as estimator.
Instructions for updating:
ModelValidator is deprecated, use Evaluator instead.
INFO:absl:Adding upstream dependencies for component BigQueryExampleGen
INFO:absl:Adding upstream dependencies for component StatisticsGen
INFO:absl:   ->  Component: BigQueryExampleGen
INFO:absl:Adding upstream dependencies for component SchemaGen
INFO:absl:   ->  Component: Sta

In [ ]:
# 4.1 Update only  command

In [1]:
# Update the pipeline
!tfx pipeline update \
--pipeline_path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT}

/bin/sh: 1: tfx: not found


In [ ]:
# 4.2 Run updated pipeline

In [43]:
# You can run the pipeline the same way.
!tfx run create --pipeline_name {PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: mch_pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: mch_pipeline
+-----------------+--------------------------------------+----------+---------------------------+
| pipeline_name   | run_id                               | status   | created_at                |
+=================+======================================+==========+===========================+
| mch_pipeline    | 86b08b26-b24f-48fd-b77f-ff37206c3535 |          | 2020-03-28T02:02:25+00:00 |
+-----------------+--------------------------------------+----------+---------------------------+
